# Exploring Dimlp and Fidex rule generation for breast cancer classification

**Introduction:**

Welcome to HES-Xplain, our interactive platform designed to facilitate explainable artificial intelligence (XAI) techniques. In this use case, we dive into breast cancer classification and showcase the power of Dimlp and Fidex as interpretability tools.

In the medical field, there are many machine-learning models that can obtain results. These models are like black boxes that return results without fully understanding the reasoning leading to them. Depending on the nature of the problem it's important to understand the model's decision. It's difficult for a doctor or a patient to make a decision based on a model that we don't understand. Likewise, insurance companies will not necessarily agree to pay for a medication that a model has advised people to take. This is why explainability in this field is important.

Fidex is an algorithm used in classification problems that allows us to obtain a rule explaining the decision class of the model for a given test sample. It enables us to grasp the model's decision-making process and to better understand the importance of each parameter in discerning the nature of the tumor, distinguishing between benign and malignant cases.

The global version of Fidex, FidexGlo, allows us to get a set of rules that englobes all the decision-making process for the entire training set. We can then check if a new sample verifies all the properties of one rule of the ruleset, in which case this rule explains the model's decision.

Our goal is to demonstrate how HES-Xplain empowers users to explore and interpret breast cancer classification effectively. By the end of this use case, you'll have a solid understanding of how to use Dimlp to train the model and the Fidex algorithms into your own classification datasets.

**Objectives:**

    1. Understand the importance of interpretability in medical problems.
    2. Introduce Dimlp and Fidex as powerful XAI techniques.
    3. Understand how to use Dimlp and Fidex.
    4. Showcase the capabilities of HES-Xplain in implementing Dimlp and Fidex algorithms.
    5. Provide practical insights into applying Dimlp and Fidex to breast cancer classifiers through an interactive notebook.
    6. Foster a community of XAI enthusiasts and practitioners.

**Outline:**

    1. Dataset and Problem Statement.
    2. Load the dataset.
    3. Training of the Model.
    4. Local rules generation - Fidex
    5. Global ruleSet generation - FidexGlo
    6. Conclusion.
    7. References.

Through this use case, we aim to empower users to grasp the potential of Dimlp and Fidex as tools for transparent and interpretable classification. With HES-Xplain, we make XAI accessible, helping users build trust in their models and make informed decisions.


# Dataset and Problem Statement

The Breast Cancer Dataset used in this use case consists of 569 data samples representing patients who have tumors. Each patient has 30 attributes, computed out of an image, used to determine if the tumor is malignant (first class) or benign (second class). This dataset is available on [Kaggle](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data) and can be imported with sklearn. It is composed of 455 training samples and 114 testing samples. 

The 30 attributes are composed of 10 real-valued features computed for each cell nucleus :

- **Radius**: Mean of distances from the center to points on the perimeter.
- **Texture**: Standard deviation of gray-scale values.
- **Perimeter**
- **Area**
- **Smoothness**: Local variation in radius lengths.
- **Compactness**: $\frac{\text{perimeter}^2}{\text{area}} - 1.0$
- **Concavity**: Severity of concave portions of the contour.
- **Concave Points**: Number of concave portions of the contour.
- **Symmetry**
- **Fractal Dimension**: "Coastline approximation" - 1.

For each tumor image, the mean, standard error, and worst errors were computed and appear as different attributes.

**Problem Statement:** Our objective is to build a robust classifier capable of accurately classifying breast cancer samples among the 2 classes. By leveraging deep learning techniques and Fidex algorithms, we aim to not only achieve high classification performance but also gain insights into the attributes that contribute to the classification decisions.

# Load the dataset
First of all, we unbuffer Python output so that it prints the result as soon as possible (you won't need to do this).

In [1]:
class Unbuffered(object):
   def __init__(self, stream):
       self.stream = stream
   def write(self, data):
       self.stream.write(data)
       self.stream.flush()
   def writelines(self, datas):
       self.stream.writelines(datas)
       self.stream.flush()
   def __getattr__(self, attr):
       return getattr(self.stream, attr)

import sys
sys.stdout = Unbuffered(sys.stdout)

We start by loading the dataset. First, we need to download the data with sklearn and shuffle it randomly.

In [2]:
from sklearn import datasets
from sklearn.utils import shuffle

datas, classes = shuffle(datasets.load_breast_cancer().data, datasets.load_breast_cancer().target)

We normalize the data so that the model learns better. We save the mean and std in a file

In [3]:
import numpy as np
means = np.mean(datas, axis=0)
print(means)
stds = np.std(datas, axis=0)
normalized_datas = (datas - means) / stds

with open("data/BCWDataset/normalisation.txt", 'w') as myfile:
    for i in range(len(means)):
        myfile.write("Attribut " + str(i) + ": mean: " + str(means[i]) + ", std: " + str(stds[i]) + '\n')
    myfile.close()

[1.41272917e+01 1.92896485e+01 9.19690334e+01 6.54889104e+02
 9.63602812e-02 1.04340984e-01 8.87993158e-02 4.89191459e-02
 1.81161863e-01 6.27976098e-02 4.05172056e-01 1.21685343e+00
 2.86605923e+00 4.03370791e+01 7.04097891e-03 2.54781388e-02
 3.18937163e-02 1.17961371e-02 2.05422988e-02 3.79490387e-03
 1.62691898e+01 2.56772232e+01 1.07261213e+02 8.80583128e+02
 1.32368594e-01 2.54265044e-01 2.72188483e-01 1.14606223e-01
 2.90075571e-01 8.39458172e-02]


Then we define test and training sets by distributing the data so that there is one-fifth of the data for the test and four-fifths for training.

In [4]:
nb_samples = len(classes)
cutoff = int((4/5)*nb_samples)
train_classes = classes[:cutoff]
train_datas = normalized_datas[:cutoff]
test_classes = classes[cutoff:]
test_datas = normalized_datas[cutoff:]

We can display the number of training and testing data and check that there is the same number of data as classes.

In [5]:
print(f"There are {len(train_datas)} train datas and {len(train_classes)} train classes")
print(f"There are {len(test_datas)} test datas and {len(test_classes)} test classes")

There are 455 train datas and 455 train classes
There are 114 test datas and 114 test classes


Finally, we save the data in text files in a specific format. We create a folder BCWDataset inside a data folder if it doesn't already exist and we save the files here.
For the classes, we encode in one hot format, with 1 on the position of the right class and 0 elsewhere.

In [6]:
import os

def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")

def save_data(datafile, datas):
    f = open(datafile, "w")
    for data in datas:
        for val in data:
            f.write(str(val)+" ")
        f.write("\n")
    f.close()

def save_class(classfile, classes, nb_classes):
    f = open(classfile, "w")
    for classe in classes:
        for i in range(nb_classes):
            if classe == i:
                f.write("1 ")
            else:
                f.write("0 ")
        f.write("\n")
    f.close()

create_folder("data/BCWDataset")

nb_classes = len(datasets.load_breast_cancer().target_names)
save_data("data/BCWDataset/datas.txt", datas)
save_data("data/BCWDataset/trainData.txt", train_datas)
save_data("data/BCWDataset/testData.txt", test_datas)
save_class("data/BCWDataset/trainClass.txt", train_classes, nb_classes)
save_class("data/BCWDataset/testClass.txt", test_classes, nb_classes)

In order to generate beautiful and understandable rules, we still have to create an attribute file with attribute and class names.<br>
For the attributes we have :

In [7]:
attributes = datasets.load_breast_cancer().feature_names
print(attributes)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


And for the classes :

In [8]:
classes = datasets.load_breast_cancer().target_names
print(classes)

['malignant' 'benign']


We store the attributes and classes in a file attributes.txt and fill the spaces with _ and apply upper case.

In [9]:
f = open("data/BCWDataset/attributes.txt", "w")
for attribute in attributes:
    f.write(attribute.upper().replace(" ", "_")+"\n")
for classe in classes:
    f.write(classe.upper().replace(" ", "_")+"\n")

f.close()

# Training of the model

We train the dimlp model. First, let's import it and display the parameters that are needed and optional for training. Dimlp is able to train the model as well as generate rules with his own algorithm. The Fidex algorithms are an improvement that we'll see later on. We choose the training with DimlpBT, which is the same as Dimlp but with bagging.
The main package name is dimlpfidex, and we need the dimlp module where DimlpBT is located. To display the parameters, we can just call dimlpBT with no parameters, or with -h or --help.

In [10]:
from dimlpfidex import dimlp
res = dimlp.dimlpBT("--help")

ModuleNotFoundError: No module named 'dimlpfidex'

Now we can run dimlpBT with the right parameters.<br>

As we see in the parameter list, we need to give the train data file(--train_data_file), the number of input neurons of the model(--nb_attributes), which is the number of attributes, and the number of output neurons(--nb_classes) which is the number of classes.<br>
For this dataset, we recommend adding a hidden layer (--hidden_layers) of size 5 or 10 in the model architecture. Here we choose 5.

In the training file, we can also have the classes, but here we've chosen to save them in a different file, so we add it too with --train_class_file.<br>
To get test predictions and statistics, we specify a test data file and test class file with --test_data_file and --test_class_file.<br>
We choose the names for the files of statistics(--stats_file), train(--train_pred_outfile) and test predictions(--test_pred_outfile), and weights(--weights_outfile).<br>
We specify with --root_folder the folder where we have all the data files and where we want to save the results. <br>

This is enough to train our model, but we can add the computation of explaining rules with the Dimlp algorithm. You will be able to compare the rules obtained with the Dimlp algorithm and with the Fidex one. We just add --with_rule_extraction to ask for the computation of the rules, --global_rules_outfile to specify the name of the rules file, and --attributes_file, to give the attribute and class names that will appear in the rules.<br>

As there are a lot of things to be written on output, we save the program output with --console_file. If you remove it you may have some neverending execution problem in the notebook. <br>

To execute dimlpBT, we can either write the whole command (see the commented line below) or call a JSON configuration file (--json_config_file) containing each parameter as we want. All configuration files are located in the data folder. You can create new ones or modify them as you like by adding, removing, or modifying some parameters. To create a new file, we recommend you use our graphical user interface (GUI). <br>

Additionally, we have a notebook that explains how to use this GUI, providing detailed instructions and practical examples. We encourage you to read this notebook to get the most out of our tools. You can access the notebook by following this link: **Link to the notebook**. <br>

Here is how we execute dimlpBT :

In [ ]:
# Creation of Weights folder if it doesn't exist
create_folder("data/BCWDataset/Weights")
#result = dimlp.dimlpBT("dimlpBT -L trainData.txt -1 trainClass.txt -T testData.txt -2 testClass.txt -A attributes.txt -w Weights/weights -I 30 -O 2 -p predTrain.out -t predTest.out -R -F dimlpBTRules.rls -o stats.txt -r resultDimlpBT.txt -S data/BCWDataset") 
result = dimlp.dimlpBT("--train_data_file trainData.txt --train_class_file trainClass.txt --test_data_file testData.txt --test_class_file testClass.txt --attributes_file attributes.txt --nb_attributes 30 --hidden_layers 5 --nb_classes 2 --weights_outfile Weights/weights.wts --with_rule_extraction true --global_rules_outfile dimlpBTRules.rls --train_pred_outfile predTrain.out --test_pred_outfile predTest.out --stats_file stats.txt --console_file resultDimlpBT.txt --root_folder data/BCWDataset")
result = dimlp.dimlpBT("--json_config_file data/config_dimlpBT.json")
if (result == 0):
    print("Training done")

If you go to the folder data/BCWDataset and open the file result.txt, you can see that the training went through, and some rules were computed. You can now open the file stats.txt in the same folder to see the train and test accuracy as well as the sum squared errors. In the folder, you can also see the train and test predictions, the weights obtained for Fidex, and the dimlpRules.<br>

In the file dimlpRules.txt, you can see the rule set, some statistics on the rules, and some statistics on the ruleset. Each rule is composed of antecedents and target class as well as the number of covering of the rule, which is the number of examples that verify(or "activate") the rule (even if the class is not correct).<br>

For each rule, we have from left to right:<br>
The number of covered samples, the number of correct covered samples, the number of false covered samples, and the accuracy of the rule.<br>

The statistics of the ruleset are: <br>
- The number of rules in the set
- The mean total and the mean number of antecedents per rule
- The covering of the rule
- The rules' accuracy
- The fidelity to the model, which is the percentage of covered samples that are correct with respect to the model's decision.
- The model accuracy if we keep only the samples for which the model and the rules decision agree
- The default rule activation rate, which is the percentage of samples for which no rule is activated

# Local rules generation - Fidex
Now we can generate some local rules to explain the models' results. We can start with launching Fidex on one test sample. This will generate a rule explaining the sample locally. It is local because the algorithm searches a rule only for one sample.
Fidex is located in the fidex module. Let's take a look at the parameters :

In [ ]:
from dimlpfidex import fidex
res = fidex.fidex("--help")

We see that we need the number of attributes and classes, the train and test data and class files, as well as train and test predictions, weights file name, and rule output file name where the rules will be saved.<br>
We specify as well the saved folder and the attributes.<br>

To see what happens, we launch it with just one sample, we save beforehand the test data sample in a file with its class and predictions:

In [ ]:
# Get data from file
def get_data(file_name):
    with open(file_name, "r") as my_file:
        data = []
        line = my_file.readline()
        while line:
            line = line.strip()  # Remove the line break at the end of the line
            di = [float(elt) for elt in line.split(" ")]
            data.append(di)
            line = my_file.readline()
        my_file.close()
    return data

test_preds = get_data("data/BCWDataset/predTest.out")

# Save the data, prediction, and class of the first sample in a new file
f = open("data/BCWDataset/testDataSample.txt", "w")
for val in test_datas[0]:
    f.write(str(val)+" ")    
f.write("\n")
for val in test_preds[0]:
    f.write(str(val)+" ")    
f.write("\n")
val = test_classes[0]
for i in range(nb_classes):
    if val == i:
        f.write("1 ")
    else:
        f.write("0 ")  
f.write("\n")
f.close()

#res = fidex.fidex("fidex -T trainData.txt -P predTrain.out -C trainClass.txt -S testDataSample.txt -W Weights/weights -N 25 -A attributes.txt -O rule.txt -R data/BCWDataset")
#res = fidex.fidex("--train_data_file trainData.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --test_data_file testDataSample.txt --weights_file Weights/weights.wts --attributes_file attributes.txt --rules_outfile rule.rls --nb_attributes 30 --nb_classes 2 --root_folder data/BCWDataset")
result = fidex.fidex("--json_config_file data/config_fidexOne.json")

You can see the walkthrough of the algorithm and the rule extracted. The rule is also saved in the rule.rls file. With the rule, we see also the covering size of the rule on the training set, the fidelity, the accuracy, and the confidence of the rule. The confidence shows how much the rule is confident with his choices, with respect to the prediction values.<br>

Now, we execute Fidex with all test samples. We send the console output in the fidexResult.txt file and save the global statistics in fidexStats.txt. **It should take about a minute**.

In [ ]:
#res = fidex.fidex("fidex -T trainData.txt -P predTrain.out -C trainClass.txt -S testData.txt -p predTest.out -c testClass.txt -W Weights/weights -N 25 -A attributes.txt -O fidexRules.rls -s fidexStats.txt -r fidexResult.txt -R data/BCWDataset")
#res = fidex.fidex("--train_data_file trainData.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --test_data_file testData.txt --test_class_file testClass.txt --test_pred_file predTest.out --weights_file Weights/weights.wts --attributes_file attributes.txt --rules_outfile fidexRules.rls --stats_file fidexStats.txt --console_file fidexResult.txt --nb_attributes 30 --nb_classes 2 --root_folder data/BCWDataset")
res = fidex.fidex("--json_config_file data/config_fidex.json")

if (res == 0):
    print("Fidex done")

You can see the rules of each sample in the file fidexRules.rls. The global statistics on the test set appear in statsFidex.txt. There is the mean covering size per rule, the mean number of antecedents per rule, and the mean rule fidelity, accuracy, and confidence.

# Global ruleSet generation - FidexGlo
We've seen how to compute a rule that explains the decision of the model for a specific sample. Now, we will generalize a ruleset that characterizes the whole train dataset. That means that for each training sample, there is a rule in the set of rules that explains the model's decision for this sample. We will use this global ruleset to explain the results obtained on new test samples. If there is a rule of the ruleset corresponding to the sample, we take this one and get a global explanation for the sample. If there is none, we call Fidex and only have a local explanation.<br>

To get the ruleSet we execute fidexGloRules which is located in the fidex module. Here are the possible parameters :

In [ ]:
res = fidex.fidexGloRules("--help")

We use nearly the same parameters as for Fidex but we only need train data. We need to choose a heuristic for fidexGloRules, we choose the optimal to get better results.<br>
**It should take about 5 minutes**. If you want to accelerate the process, you can use some dropout, the algorithm will randomly skip some dimensions or some hyperplans. For example, you can put: -d 0.5 -h 0.5, to skip half dimensions and half hyperplans, which should be about 4 times faster.

In [ ]:
#res = fidexGlo.fidexGloRules("fidexGloRules -T trainData.txt -P predTrain.out -C trainClass.txt -W Weights/weights -N 25 -A attributes.txt -O globalRules.rls -M 1 -r fidexGloRulesResult.txt -S data/BCWDataset")
#res = fidex.fidexGloRules("--train_data_file trainData.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --weights_file Weights/weights.wts --nb_attributes 30 --nb_classes 2 --heuristic 1 --global_rules_outfile globalRules.rls --console_file fidexGloRulesResult.txt --root_folder data/BCWDataset")
res = fidex.fidexGloRules("--json_config_file data/config_fidexGloRules.json")
if (res == 0):
    print("FidexGloRules done")

You can see the console result in the file fidexGloRulesResult.txt and the ruleset in the file globalRules.rls.
The algorithm is random, so each execution may generate different rules and a different number of them. It should generate about 20 rules. You can see at the top of the globalRules file the number of rules, the mean covering number per rule, and the mean number of antecedents. Here is an example of a rule that you may obtain:<br>

Rule 1: MEAN_CONCAVE_POINTS<0.041282 WORST_CONCAVE_POINTS<0.150829 -> BENIGN <br>
Train Covering size : 249 <br>
Train Fidelity : 1 <br>
Train Accuracy : 0.959839 <br>
Train Confidence : 0.965581 <br>

This rule is the first rule, which means that it's the rule with the maximum covering. Here, 249 train samples verify this rule. She is 100% fidel with the model and has about 96% train accuracy.
This rule says that if you have mean concave points lower than 0.41282 and worst concave points lower than 0.15089, then your breast cancer is benign. And this rule is accurate, on train test, at 96%.<br>

Now, we can see some global statistics on the test set, and also some statistics directly on each rule. So we will see the test accuracy on this rule.<br>
We execute fidexGloStats which is located in the fidex module. First, let's check the parameters :

In [ ]:
res = fidex.fidexGloStats("--help")

We need the test files(data, prediction, class), the rules file, and the number of attributes and classes. We need to specify the attribute file as well. We choose to save the results in the file fidexGloStats.txt. With --global_rules_outfile we can generate the statistics on rules which will modify the rules file. If you want to keep the first ruleSet unchanged, you should give another name. Finally, with --positive_class_index we can specify the positive class in order to get a ROC curve and some other statistics on false and true positives/negatives. We will design the malignant class as positive. So here is the command :

In [ ]:
#res = fidexGlo.fidexGloStats("fidexGloStats -T testData.txt -P predTest.out -C testClass.txt -R globalRules.rls -F globalRules.rls -A attributes.txt -O fidexGloStats.txt -x 0 -S data/BCWDataset")
res = fidex.fidexGloStats("--test_data_file testData.txt --test_pred_file predTest.out --test_class_file testClass.txt --attributes_file attributes.txt --global_rules_file globalRules.rls --global_rules_outfile globalRules.rls --nb_attributes 30 --nb_classes 2 --stats_file fidexGloStats.txt --positive_class_index 0 --root_folder data/BCWDataset")
res = fidex.fidexGloStats("--json_config_file data/config_fidexGloStats.json")

Here you can see the global statistics on the test set. As the previous calculations have randomness, each execution is a bit different. So here we will discuss the same case as before, so you may have slightly different results. In our case, we had about 95% fidelity, which is good, and a rule accuracy about 1% lower than the model accuracy. So the rules seem to classify a bit worse. The explainability rate is the percentage of samples for which we can find a rule in the rules set. For the others, we need to execute Fidex (this is the default rule rate). In our case, we had more than a 97% explainability rate, so only in 3% of cases do we need to compute Fidex. Each rule can activate many rules. Here on average, a sample activates 3 correct rules and 0.1 wrong rules. A wrong rule is a rule with which the model doesn't agree. For example, if the rule says malign and the model says benign. Something interesting is the model test accuracy when rules and model agree. You can see that, generally, the accuracy increases if we consider samples where rules and model agree, and increases even more if we take only the activated rules (when there are no activated rules, we choose the model prediction). That means that the rules confirm well the model decision, but when no rule is found, the model decision may as well be wrong. <br>

Finally, we have the statistics on the positive/negative decisions of the model and of the rules. For the model's decisions, we had 1% false positives and 7% false negatives. That means that when the model says that it's malign, it has a 99% chance of being malign. But when it says that it's benign, there is a 7% risk that it is malign in reality. So the precision is 97% and the recall is 93%.

In the case of the rules decisions, the rules decision is the same as the model if we find a correct rule. If there is no rule, we can launch Fidex, so it's also the same decision as the model because it will create a fidel rule. The only scenario when the decision changes from the model's is when some rules are activated, all rules decide the same class which is not the class chosen by the model. The results are really close with 95% of precision and 93% of recall.

In globalRules file, you can now see the statistics of rules on the test set. Here is the same rule as before that I have now:<br>

Rule 1: MEAN_CONCAVE_POINTS<0.041282 WORST_CONCAVE_POINTS<0.150829 -> BENIGN <br>
Train Covering size : 249 --- Test Covering size : 65 <br>
Train Fidelity : 1 --- Test Fidelity : 0.969231 <br>
Train Accuracy : 0.959839 --- Test Accuracy : 0.984615 <br>
Train Confidence : 0.965581 --- Test Confidence : 0.963119 <br>

You can see that the rule no longer always agrees with the model, only in 0.969% of cases. However, the rule accuracy has increased with the test. That means that the rule is very good in reality, with 98.4% of correct classification.

In [ ]:
import sys
print(sys.path)
from trainings.computeRocCurve import computeRocCurve
res = computeRocCurve("--help")

We execute computeRocCurve :

In [ ]:
#res = computeRocCurve(test_class = "testClass.txt", test_pred = "predTest.out", positive_index = 0, output_roc="outRoc.png", stats_file="stats.txt", save_folder = "data/BCWDataset")
#res = computeRocCurve('--test_class_file testClass.txt --test_pred_file predTest.out --positive_class_index 0 --output_roc outRoc.png --stats_file rocStats.txt --root_folder data/BCWDataset --nb_classes 2')
res = computeRocCurve("--json_config_file data/config_roc.json")

The AUC score is added to the stats file and the ROC curve is saved in the outRoc.png file. We get an AUC of about 99%. <br>
We can visualize the AUC curve :

In [ ]:
from IPython.display import Image, display

display(Image(filename="data/BCWDataset/outRoc.png"))

Finally, it's possible to launch fidexGlo on a test set. It will, for each test sample, search in the global ruleset for all the rules that verify the sample's properties and match the decision of the model. If no rule verifies the properties, it calls Fidex on the test sample. In the other case, it gives all the correct and wrong activated rules.<br>

FidexGlo is located in the fidex module. Here are the possible parameters:

In [ ]:
res = fidex.fidexGlo("--help")

As we want to use Fidex on test sample not covered by the global ruleset, we need to specify the Fidex parameters. We launch it with :

In [ ]:
#res = fidexGlo.fidexGlo("fidexGlo -S testData.txt -p predTest.out -R globalRules.rls -A attributes.txt -O explanations.txt -r fidexGloResults.txt -F data/BCWDataset")
#res = fidex.fidexGlo("--test_data_file testData.txt --test_pred_file predTest.out --global_rules_file globalRules.rls --nb_attributes 30 --nb_classes 2 --attributes_file attributes.txt --explanation_file explanations.txt --console_file fidexGloResults.txt --root_folder data/BCWDataset --with_fidex true --train_data_file trainData.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --test_class_file testClass.txt --weights_file Weights/weights.wts")
res = fidex.fidexGlo("--json_config_file data/config_fidexGlo.json")

if res == 0:
    print("Explanations generated") 

The explanations for each test sample can then be found in the file explanations.txt.

# Conclusion

Throughout this notebook, we've learned how to use DimlpBT to train a dataset and get some explaining rules, and we got into the different Fidex algorithms to get some local and global explanations, and some statistics on the model's decisions. We used the example of the breast cancer dataset from Wisconsin and we saw that the algorithms perform very well. It gives some good explanations of the model's decision and does it very fast, especially if a fidexGlo ruleset has been computed beforehand. You can try with any tabular dataset you want, you just have to remember to transform the data in a good format. <be>

We didn't speak about every single parameter, you can try to change every hyperparameter you want to see how it goes. An interesting parameter that we didn't consider is the parameter for decision threshold (--decision_threshold) in Fidex algorithms. It allows us to change prediction with respect to a specific threshold on the positive class. If the model gives a score prediction, for the positive class, greater than this threshold, the model prediction is considered to be the positive class, even if another class obtains a higher score. If the recall is not good enough, it's possible to improve it this way. Another parameter is the seed. It allows you to remove the randomness of the algorithms and get the same result in each execution for the same parameters and data.<br>

We considered only a simple tabular dataset. It's also possible to use it for an image classification problem. To see how we can train a dataset with convolutions, we recommend exploring the Mnist notebook for a hands-on experience **Insérer lien notebook Mnist**. There are also other training methods, like an MLP or decision trees. The Keel Australian notebook goes through these and shows what to do when we have categorical attributes **Insérer Keel Australian notebook**.

# Références

Article DimlpBT de Guido
Article Fidex de Guido (à venir)